# data_cleansing_tutorial
Data cleansing tutorial in Pandas for Chipy scientific SIG

## Data Import

In [10]:
import pandas as pd

In [11]:
df14 = pd.read_csv('../data/Immunization_14-15.csv')
df09 = pd.read_csv('../data/immunization_09-10.csv')
df04 = pd.read_csv('../data/immunization_04-05.csv')

### Initial data verification
Let's first take a look at the data briefly to make sure it's in the format we want.

In [12]:
df04.head()

,Immunization School Survey Report 2004-05,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,RCDT,Region,District,School,Total Sch. Enroll.,Total - NC,Unduplic.,Total Excluded,NC-PE Only,Polio Prot.,...,Cpox Prot.,Cpox Rel.,Cpox Med.,Cpox AS,Cpox UP/NC,Cpox Total,NaN,NaN,NaN,NaN
1,Adams County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01001001026,01,0010,0001,279,2,2,2,0,277,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,01001002026,01,0020,2002,278,1,1,1,0,276,...,138,0,0,1,1,140,NaN,NaN,NaN,NaN
4,01001002026,01,0020,2002,419,0,0,0,0,419,...,161,0,0,0,0,161,NaN,NaN,NaN,NaN


Here we see that the header information is wrong. The correct header is row 0. Because we are lazy, we will re-import the data and drop the first row instead of formatting this current version.

In [20]:
df04 = pd.read_csv('../data/immunization_04-05.csv', skiprows=1)
df04.head()

,RCDT,Region,District,School,Total Sch. Enroll.,Total - NC,Unduplic.,Total Excluded,NC-PE Only,Polio Prot.,...,Cpox Prot.,Cpox Rel.,Cpox Med.,Cpox AS,Cpox UP/NC,Cpox Total,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62
0,Adams County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01001001026,1,0010,0001,279,2,2,2,0,277,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,01001002026,1,0020,2002,278,1,1,1,0,276,...,138,0,0,1,1,140,NaN,NaN,NaN,NaN
3,01001002026,1,0020,2002,419,0,0,0,0,419,...,161,0,0,0,0,161,NaN,NaN,NaN,NaN
4,01001002X06,1,002X,n/a,197,0,0,0,0,193,...,101,2,0,0,0,103,NaN,NaN,NaN,NaN


Cool, the columns are correct but per our goals we only need the columns for Polio. Let's remove the rest.

In [21]:
columns_to_keep = ['RCDT', 'School', 'Total Sch. Enroll.', 'Polio Prot.']
df04 = df04.ix[:, columns_to_keep] # takes all rows, only the columns in the array columns_to_keep
df04.head()

,RCDT,School,Total Sch. Enroll.,Polio Prot.
0,Adams County,NaN,NaN,NaN
1,01001001026,0001,279,277
2,01001002026,2002,278,276
3,01001002026,2002,419,419
4,01001002X06,n/a,197,193


Interesting, the first row has NaN. Looking back at the table above, this is because the original table has some extra rows for Counties e.g. "Adams County".

Let's simply drop the NaNs in this table.

In [24]:
df04.dropna(inplace=True) # inplace=True replaces the previous data frame.
                          # This is the same as df04 = df04.dropna(inplace=False)
df04.head()

,RCDT,School,Total Sch. Enroll.,Polio Prot.
1,01001001026,0001,279,277
2,01001002026,2002,278,276
3,01001002026,2002,419,419
4,01001002X06,n/a,197,193
5,01001003026,1001,330,329


Instead of looking just the first 5 rows, let's check out some summary statistics.

#### Summary Statistics

In [25]:
df04.describe()

,RCDT,School,Total Sch. Enroll.,Polio Prot.
count,5179,5179,5179,5179
unique,2045,701,1237,1240
top,15016299025,n/a,231,127
freq,594,1165,19,16


In [26]:
df04.dtypes

RCDT                  object
School                object
Total Sch. Enroll.    object
Polio Prot.           object
dtype: object

Uh oh, the two columns are supposed to be numerical, not strings (denoted here as objects).

Let's convert these now.

In [33]:
# inplace isn't available for this function, thus asign manually
cols = ['Total Sch. Enroll.', 'Polio Prot.']
df04[cols] = df04[cols].applymap(lambda x: int(x))
df04.head()

ValueError: ("invalid literal for int() with base 10: '1,628'", u'occurred at index Total Sch. Enroll.')

This error shows that there's are values with commas, so Python can't automatically change it into an int.

We have to remove the comma, then convert to int.

In [ ]:
df04[cols] = df04[cols].applymap(lambda x: int(x.replace(',', '')))
df04.head()

In [37]:
df04.dtypes

RCDT                  object
School                object
Total Sch. Enroll.     int64
Polio Prot.            int64
dtype: object

Beautiful. Now let's run those summary stats again.

In [39]:
df04.describe(include='all')

,RCDT,School,Total Sch. Enroll.,Polio Prot.
count,5179,5179,5179.000000,5179.000000
unique,2045,701,NaN,NaN
top,15016299025,n/a,NaN,NaN
freq,594,1165,NaN,NaN
mean,NaN,NaN,454.953659,450.193281
std,NaN,NaN,459.038458,455.747560
min,NaN,NaN,1.000000,1.000000
25%,NaN,NaN,191.000000,189.000000
50%,NaN,NaN,344.000000,341.000000
75%,NaN,NaN,561.000000,554.500000


In [40]:
df04['Total Sch. Enroll.'].hist()

RuntimeError: Python is not installed as a framework. The Mac OS X backend will not be able to function correctly if Python is not installed as a framework. See the Python documentation for more information on installing Python as a framework on Mac OS X. Please either reinstall Python as a framework, or try one of the other backends. If you are Working with Matplotlib in a virtual enviroment see 'Working with Matplotlib in Virtual environments' in the Matplotlib FAQ